In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [7]:
pip install --upgrade pandas pyarrow


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install fastparquet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [10]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [11]:
num_columns = len(df.columns)
print(f"Number of columns: {num_columns}")

Number of columns: 19


In [12]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [13]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [14]:
std_duration = df['duration'].std()
print(f"Standard deviation of trip durations for January: {std_duration}")

Standard deviation of trip durations for January: 42.59435124195457


In [15]:
original_count = len(df)
print(f"Original Count: {original_count}")

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
filtered_count = len(df_filtered)
print(f"Filtered Count: {filtered_count}")

fraction_remaining = filtered_count / original_count
print(f"Fraction of records remaining: {fraction_remaining:.2%}")

Original Count: 3066766
Filtered Count: 3009173
Fraction of records remaining: 98.12%


In [16]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [17]:
train_dicts = df[categorical].to_dict(orient='records')
# train_dicts

In [18]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [19]:
unique_pulocation_ids = df['PULocationID'].nunique()
unique_dolocation_ids = df['DOLocationID'].nunique()

print(f"Unique PULocationID values: {unique_pulocation_ids}")
print(f"Unique DOLocationID values: {unique_dolocation_ids}")

Unique PULocationID values: 255
Unique DOLocationID values: 260


In [20]:
num_columns = X_train.shape[1]
print(f"Number of columns in the feature matrix: {num_columns}")

Number of columns in the feature matrix: 515


In [21]:
y_train = df['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [22]:
print(f"RMSE of the model on the training data: {root_mean_squared_error(y_train, y_pred)}")

RMSE of the model on the training data: 7.649261827760637


In [23]:
# Step 1: Load validation dataset
val_df = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

# Step 2: Process categorical columns and ensure they are strings
categorical = ['PULocationID', 'DOLocationID']
val_df[categorical] = val_df[categorical].astype(str)

# Step 3: Calculate duration in minutes for validation data
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Step 4: Filter the validation dataset to remove outliers (keep durations between 1 and 60 minutes)
val_df = val_df[(val_df['duration'] >= 1) & (val_df['duration'] <= 60)]

# Step 5: Transform the validation dataset using the trained DictVectorizer
val_dicts = val_df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)  # Use existing DictVectorizer from previous step

# Step 6: Get actual and predicted duration values
y_val = val_df['duration'].values
y_val_pred = lr.predict(X_val)  # Use trained Linear Regression model from training step

# Step 7: Calculate RMSE on validation data
val_mse = mean_squared_error(y_val, y_val_pred)  # Mean Squared Error
val_rmse = np.sqrt(val_mse)  # Root Mean Squared Error
print(f"RMSE on validation data: {val_rmse}")

RMSE on validation data: 7.811821356741418
